# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [1]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size. 


In [2]:
def merge(left, right, symbols_dict):
    left_keys = list(left[0])
    right_keys = list(right[0])

    for key in left_keys:
        symbols_dict[key].append('0')
    for key in right_keys:
        symbols_dict[key].append('1')

    merged_keys = left[0]+right[0]
    merged_values = left[1]+right[1]

    return (merged_keys, merged_values)
    

def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities  
    
    Arguments:
    ----------
    probability_dict:
      - keys: symbol as character or string
      - values: probability of the symbol as a float or double
      Example: {"A": 0.25, "B":0.5,"C":0.125,"D":0.125}
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """

    # Convert the dictionary to a list of tuples
    probability_list = list(probability_dict.items())

    symbols_dict = {}
    for key in probability_dict.keys():
        symbols_dict[key] = []

    for i in range(len(probability_list)):
        probability_list[i] = ([probability_list[i][0]], probability_list[i][1])

    trees = []
    for i in range(len(probability_list)):
        trees.append(probability_list[i])

    # merge the trees until there is only one tree left by using the merge function
    while len(trees) > 1:
        trees.sort(key=lambda x: x[1])
        left = trees.pop(0)
        right = trees.pop(0)
        merged = merge(left, right, symbols_dict)
        trees.append(merged)

    # reverse the order of the codewords and convert the list to a string
    for key in symbols_dict.keys():
        symbols_dict[key] = symbols_dict[key][::-1]
        symbols_dict[key] = ''.join(symbols_dict[key])

    return symbols_dict

In [3]:
prob_dict = {"A":0.05,"B":0.10,"C":0.15,"D":0.15,"E":0.2,"F":0.35}
print(Huffman_code(prob_dict))

{'A': '000', 'B': '001', 'C': '100', 'D': '101', 'E': '01', 'F': '11'}


### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course. 

In [4]:
def LZ_online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols  
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    
    Return:
    -------
    - dictionary : the computed dictionnary in the form:
        - keys: symbol as character or string
        - values: associated codeword as a tuple composed of the entry index (integer) and a binarized adress with one appended symbol (character or string)
        Example: {'': (0, ''), '0': (1, '0'), '1': (2, '01'), '00': (3, '010'), '10': (4, '100')}
    - encoded_sequence : the encoded sequence in the string format
    """
    dict = {'': (0, '')}
    encoded = ''
    next_index = 1
    i = 0

    while i < len(sequence):

        prefix = ''
        buffered_prefix = ''
        j = i
        next_index_log = np.ceil(np.log2(next_index))

        while prefix in dict and j < len(sequence):
            buffered_prefix = prefix
            prefix += sequence[j]
            j += 1

        if prefix not in dict and len(prefix) == 1:

            prefix = buffered_prefix
            prefix_dict = dict[prefix]
            index = '{0:b}'.format(prefix_dict[0])
            encoded += sequence[j - 1].zfill(int(next_index_log + 1))

            dict[prefix + sequence[j - 1]] = (next_index, sequence[j - 1].zfill((int(next_index_log + 1))))
            next_index += 1
            i = j

        if prefix not in dict and len(prefix) > 1:
            prefix = buffered_prefix
            prefix_dict = dict[prefix]
            index = '{0:b}'.format(prefix_dict[0])
            encoded += index.zfill(int(next_index_log)) + sequence[j - 1]

            dict[prefix + sequence[j - 1]] = (next_index, index.zfill(int(next_index_log)) + sequence[j - 1])
            next_index += 1
            i = j
        else:
                i = j

    return dict, encoded

In [5]:
t_seq = '1011010100010'
u_seq = '100011101100001000010'

dict, u_lz = LZ_online(t_seq)
print(u_lz)
print(u_seq == u_lz)

print(len(t_seq)*8)
print(len(u_lz))

100011101100001000010
True
104
21


### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [6]:
def best_match(sliding_window, look_ahead_buffer):
    """
    Find the best match between the sliding window and the look ahead buffer
    
    Arguments:
    ----------
    - sliding_window : sliding window as a string
    - look_ahead_buffer : look ahead buffer as a string
    
    Return:
    -------
    - best_match : tuple composed of the length of the match (integer) and the index of the match in the sliding window (integer)
    """
    d = 0 # distance to the start of prefix
    p = 0 # length of prefix
    c = look_ahead_buffer[0] # next character in the look ahead buffer
    i = 0

    while i < len(sliding_window):
        substring = look_ahead_buffer[:i+1]
        if substring in sliding_window:
            d = len(sliding_window) - sliding_window.rfind(substring)
            p = len(substring)
            if p + 1 > len(look_ahead_buffer):
                c = None
            else:
                c = look_ahead_buffer[i + 1]
        else :
            break
        i += 1

    if c is not None:
        best_match = str(d) + str(p) + c
    else:
        best_match = str(d) + str(p)

    return best_match, p+1


def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size

    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer

    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """
    encoded_sequence = ''
    sliding_window = ''
    look_ahead_buffer = sequence
    
    while len(look_ahead_buffer) > 0:
        match, slide = best_match(sliding_window, look_ahead_buffer)
        encoded_sequence += match
        sliding_window += look_ahead_buffer[:slide]
        look_ahead_buffer = look_ahead_buffer[slide:]
        sliding_window = sliding_window[-window_size:]

    return encoded_sequence

In [7]:
sequence = 'abracadabra'
print(LZ77(sequence,7))

sequence = 'abracadabrad'
print(LZ77(sequence,7))

sequence = 'abracadabraa'
print(LZ77(sequence,7))

sequence = 'abracadabraaa'
print(LZ77(sequence,7))

00a00b00r31c21d74
00a00b00r31c21d74d
00a00b00r31c21d74a
00a00b00r31c21d74a11


In [8]:
# [Locked Cell] Evaluation of your functions by the examiner. 
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression

Source coding and reversible (lossless) data compression
An image is a visual representation of an object or scene that can be captured and stored in a digital format. For grayscale images, each pixel is assigned a brightness value between 0 and 255 to represent the different shades of gray. One popular format for storing and transmitting grayscale images is the Portable Network Graphics (PNG) format, which is a lossless compression format. This allows PNG images to be smaller in size than raw image formats without sacrificing image quality. In this assignment, you are given a pixel.txt containing the original raw grayscale image pixels of size 512x512 encoded in bytes, where the pixels are ordered from left to right and top to bottom. Additionally, you are provided with the same image that has been encoded in the PNG format and transformed in bytes. For simplicity each byte is represented in the files with its corresponding hexadecimal representation.

### Question 5

Write a function to read and display both images (you may use external libraries). What is the number of symbols required to represent all possible images in both image representations? What is the length (in bytes) of both files?

In [11]:
from PIL import Image
from io import BytesIO

print("Initial number of symbols : ", 512*512)

raw_path = "data/pixel.txt"
png_path = "data/PNG.txt"

with open(raw_path, "r") as raw:
    raw_data = raw.read()

raw_data.replace('\r\n', ' ')
raw_data = bytes.fromhex(raw_data)
print("Length of the pixel file ", len(raw_data))

raw_img = Image.frombytes("L", (512, 512), raw_data)
raw_img.show(title="Raw image")
raw_img.save("data/raw.png")

with open(png_path, "rb") as png:
    png_data = png.read()

png_data = bytes.fromhex(png_data.decode("utf-8"))
print("Length of the PNG file ", len(png_data))

png_img = Image.open(BytesIO(png_data))
png_img.show(title="PNG image")
png_img.save("data/png.png")

Initial number of symbols :  262144
Length of the pixel file  262144
Length of the PNG file  160552


### Question 6

Estimate the marginal probability distribution of all symbols (in hexadecimal representation) from the given PNG representation sequence of the image, and determine the corresponding binary Huffman code and the encoded PNG sequence. Give the total length of the encoded PNG sequence and the compression rate.

In [49]:
with open("data/PNG.txt", "rb") as png:
    png_data = png.read().replace(b'\r\n', b'')
    png_data = png_data.decode("utf-8")

png_data = png_data.replace(" ", "")
png_data = np.array([png_data[i:i+2] for i in range(0, len(png_data), 2)])
length_data = len(png_data)
print("Length of the PNG file (bits):", length_data*8)

unique, counts = np.unique(png_data, return_counts=True)

dict = {}
for i in range(len(unique)):
    dict[unique[i]] = counts[i]/length_data

print("Number of unique symbols in the PNG file:", len(dict))
huffman_code = Huffman_code(dict)
print("Huffman code for the PNG file: ", huffman_code)

encoded = ''
for i in range(len(png_data)):
    encoded += str(huffman_code[png_data[i]])

print("Length of the encoded PNG file (bits):", len(encoded))
print("Compression ratio:", len(encoded)/(length_data*8))

Length of the PNG file (bits): 1284416
Number of unique symbols in the PNG file: 256
Huffman code for the PNG file:  {'00': '1111011', '01': '1000000', '02': '0000100', '03': '0000001', '04': '11111000', '05': '11100011', '06': '11100110', '07': '11111010', '08': '11101111', '09': '10001110', '0A': '10100010', '0B': '10111001', '0C': '11100010', '0D': '10111011', '0E': '11100001', '0F': '11101010', '10': '11101101', '11': '01001000', '12': '01110011', '13': '01110111', '14': '10111100', '15': '10000011', '16': '01101010', '17': '10111110', '18': '11101011', '19': '10100011', '1A': '00101000', '1B': '01101101', '1C': '11011001', '1D': '01111010', '1E': '10001010', '1F': '11011111', '20': '11110000', '21': '00111111', '22': '01110001', '23': '111110110', '24': '10001011', '25': '01010101', '26': '01111011', '27': '01101000', '28': '11000011', '29': '00100011', '2A': '00100100', '2B': '00010110', '2C': '10110000', '2D': '00101111', '2E': '10011010', '2F': '01001001', '30': '11100000', '31

### Question 7

Give the expected average length for your Huffman code. Compare this value with (a) the empirical average length, and (b) theoretical bound(s). Justify.

### Question 8

Plot the evolution of the empirical average length of the encoded PNG using your Huffman code for increasing input sequence lengths.

### Question 9

Encode the PNG sequence using the on-line Lempel-Ziv algorithm. Give the total length of the encoded sequence and the compression rate.

### Question 10

Encode the PNG sequence using the LZ77 algorithm with window_size=7. Give the total length of the encoded sequence and the compression rate.

### Question 11

Famous data compression algorithms combine the LZ77 algorithm and the Huffman algorithm. Explain two ways of combining those algorithms and discuss the interest of the possible combinations.

### Question 12

Encode the PNG using one of the combinations of LZ77 and Huffman algorithms you proposed in the previous question. Give the total length of the encoded PNG sequence and the compression rate.

### Question 13

Report the total lengths and compression rates using (a) LZ77 and (b) the combination of LZ77 and Huffman, to encode the PNG sequence for different values of the sliding window size (use sliding window sizes from 1 to 11000 with a step of 1000). Compare your result with the total length and compression rate obtained using the on-line Lempel-Ziv algorithm. Discuss your results.

### Question 14

Instead of encoding the PNG sequence, encode directly the pixel values with the binary Huffman algorithm. Give the average expected length, the experimental length of the encoded text and the compression rate.

### Question 15

Compare the values found at the previous question with the ones found in Question 5. In particular, is it better to first encode the text with PNG code before the Huffman encoding or to directly encode the pixel image with Huffman? Discuss.

## Channel coding

Consider a text document that is sent through a noisy communication channel. This document may contain important information such as business reports, academic papers, or personal messages. The text file may be encoded using ASCII, which uses 8 bits per character. This means that each character in the text document can be represented by a sequence of 8 binary digits, or bits. The channel used for transmission is a binary symmetric channel, which means that the transmitted bits can be flipped with a certain probability of error. The probability of error for this channel is 0.02, which means that 2% of the transmitted bits are expected to be flipped during transmission. This can result in errors in the received text, which can lead to misunderstandings or loss of important information. Therefore, it is important to develop techniques to ensure the accurate and reliable transmission of the text document over the noisy communication channel.

### Question 16

Implement a function to read the text document and encode the text signal using the binary ASCII fixed-length binary code. Count the number of bits required for the text provided with this assignment (text.txt).

### Question 17

Simulate the channel effect on the binary text signal. Then decode the text signal and display the decoded text. What do you notice?

### Question 18

Instead of sending directly through the channel the binary text signal, you will first introduce some redundancy. To do that, implement a function that returns the Hamming (7,4) code for a given sequence of binary symbols. Then, using your function, encode the binary text signal (from question 16).

### Question 19

Simulate the channel effect on the binary text signal with redundancy. Then decode the binary text signal. Display the decoded text. What do you notice? Explain your decoding procedure.

### Question 20

Given the text document encoded in binary ASCII, implement a Python program to simulate transmission over a binary symmetric channel with a noise probability ranging from 0 to 0.5, with a step of 0.01. Compute the per character error rate for each noise probability, both with and without Hamming (7,4) code. Plot the per character error rate as a function of the noise probability, and compare the performance with and without Hamming (7,4) code.

### Question 21

How would you proceed to reduce the probability of errors and/or to improve the communication efficiency? Justify.